In [1]:
from numpy import *
from matplotlib.pyplot import *
import scipy.linalg
%matplotlib inline
matplotlib.pylab.rcParams['figure.figsize']= 16,5

In [2]:
def load_data():
    """
Function to load data
    :return: data vector, data mean, data standar deviation 
    """
    data = np.genfromtxt('Resources/data.csv', delimiter=',')
    d_mean = np.mean(data)
    d_std = np.std(data)
    data = (data - d_mean) / d_std
    return data, d_mean, d_std

In [3]:
def generate_win_matrix(inSize, resSize, scaleFactor=1):
    """
Function to generate W in matrix with a uniform distribution between [-1,1]
in other Function the uniform distribution can be moved to any range
    :param inSize: number of inputs without interceptor
    :param resSize: size of reservoir
    :param scaleFactor: Scale Factor for the matrix by default 1
    :return: W in Matrix
    """
    Win = (random.uniform(low=-1, high=1, size=(resSize, 1 + inSize))) * scaleFactor
    return Win

In [4]:
def generate_w_matrix(resSize):
    """
Function to generate W matrix with a uniform distribution between [-1,1]
 In other function the uniform distribution can be moved to any range
    :param resSize: size of reservoir
    :return: W matrix
    """
    W = random.uniform(low=-1, high= 1, size = (resSize,resSize))
    return  W

In [5]:
def get_spectral_radius(W):
    """
Function to get the spectral radius of W matrix
    :param W: W matrix
    :return: spectral radius
    """
    rhoW = max(abs(linalg.eig(W)[0]))
    return  rhoW

In [6]:
def scale_w(W,rhoW, SRS = 1):
    """
Function to apply spectral radius scale factor
    :param W: W matrix
    :param rhoW: spectral radius
    :param SRS: spectral radius scale factor by default 1
    :return: W
    """
    W*=SRS/ rhoW
    return W

In [7]:
def get_x_yt_matrix(inSize, resSize,trainLen,initLen):
    """
Function to get X and Yt Matrix to save activation states and prediction targets
    :param inSize: size of the input 
    :param resSize: size of reservoir
    :param trainLen: length of train data
    :param initLen:  length of initial data
    :return: X, Yt
    """
    X = zeros((1+inSize+resSize,trainLen-initLen))
    Yt = data[None,initLen+1:trainLen+1]
    return X,Yt

In [8]:
def get_activation_states(data,resSize,trainLen,initLen,Win,W,a,X):
    """
Function to get the activations states from the reservoir
    :param data: train data
    :param resSize: reservoir size
    :param trainLen: length of train data
    :param initLen: inital data to be ignored
    :param Win: W in Matrix
    :param W: W matrix
    :param a: leaking rate
    :param X: matrix of Activations per input
    :return: last activation vector, matrix of activations X
    """
    x = zeros((resSize,1))
    for t in range(trainLen):
        u = data[t]
        x_u = tanh( dot( Win, vstack((1,u)) ) + dot( W, x ) )
        x = (1-a)*x + a*x_u 
        if t >= initLen:
            X[:,t-initLen] = vstack((1,u,x))[:,0]
    return x,X

In [9]:
def train_output_l2(inSize,resSize,X,Yt,reg = 1e-8):
    """
Function to train readout with ridge regression to address overfitting
    :param inSize: size of input
    :param resSize: reservoir size
    :param reg: penalization parameter
    :param X: Matrix of activations
    :param Yt: target values
    :return Wout: weights from reservoir and input to output layer
    """
    X_T = X.T 
    Wout = dot( dot(Yt,X_T), linalg.inv( dot(X,X_T) + reg*eye(1+inSize+resSize) ) )
    return Wout

In [10]:
def get_predictions(data,outSize,trainLen,testLen,a,Win,W,Wout,x):
    """
Function to get the predictions of the normalized data
    :param data: train data
    :param outSize: output size
    :param trainLen: length of train data
    :param testLen: length of test data
    :param a: leaking rate
    :param Win: W in Matrix
    :param W: W matrix
    :param Wout: W out Matrix
    :param x: last activations states
    :return: vector of predictions
    """
    Y = zeros((outSize,testLen))
    u = data[trainLen]
    for t in range(testLen):
        x = (1-a)*x + a*tanh( dot( Win, vstack((1,u)) ) + dot( W, x ) )
        y = dot( Wout, vstack((1,u,x)) )
        Y[:,t] = y
        # generative mode, this take the prediction and use it to make other prediction:
        u = y
        ## this would be a predictive mode, this made a prediccion and save it, then use the true value to make a new prediction:
        #u = data[trainLen+t+1]
    return Y

In [11]:
def denormalize_predictions(d_mean,d_std,Y):
    """
Function to denormalize predictions
    :param d_mean: data mean
    :param d_std: data standard deviation
    :param Y: predictions
    :return: denormalized predictions
    """
    Y1 = (Y * d_std) + d_mean
    return Y1

In [12]:
data,d_mean,d_std = load_data()
data[-12:],d_mean,d_std

(array([-1.23344482,  1.28902541, -0.44953869,  0.13451019, -0.19341094,
        -0.0944525 , -0.37580495, -0.69208391, -0.72895078,  0.62348134,
        -0.73477186, -0.44371761]), 1.2667777777777778, 5.153678260939845)

In [14]:
trainLen =len(data) - 6
testLen = 6
initLen = 12
# generate the ESN reservoir
inSize = outSize = 1
resSize = 10
a = 0.95 # leaking rate
random.seed(42)

In [15]:
Win = generate_win_matrix(inSize,resSize)

In [16]:
W = generate_w_matrix(resSize)

In [17]:
rhoW = get_spectral_radius(W)

In [18]:
W = scale_w(W,rhoW)

In [19]:
X,Yt = get_x_yt_matrix(inSize,resSize,trainLen,initLen)

In [20]:
x,X = get_activation_states(data,resSize,trainLen,initLen,Win,W,a,X)

In [21]:
Wout = train_output_l2(inSize,resSize,X,Yt)

In [22]:
Y = get_predictions(data,outSize,trainLen,testLen,a,Win,W,Wout,x)

In [23]:
Y2 = denormalize_predictions(d_mean,d_std,Y)

In [26]:
Y2

array([[ 0.57448863,  0.93559919,  0.57343448,  0.48757914,  0.83481416,
         0.7449379 ]])